# AI hjelpeverktøy for svar på rettsspørsmål relatert til transportklagenemnda

*Test- og analyseverktøy for eksperimentering og testing av løsningens ytelse*

## Initialisering

In [1]:
import openai
import os

from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.chat_models import ChatOpenAI

/Users/jens/dev/uio-rettsstat-gpt-test/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Henter personlig openAI api-nøkkel fra en .env-fil.

In [2]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

Setter path til mappen som inneholder dokumentene som skal brukes.

In [3]:
DATA_PATH = 'data/'

## Dataprossesering

### Konverter pdf til txt

Tester tika på en enkelt fil for å konvertere pdf til txt. Gjør litt cleanup for å få enklere tekst å jobbe med.

Fjerner først alt etter kategorien "Nemndas representanter". Her står det listet opp navn med de som var med i nemnda i den aktuelle saken.

Der det er sideskift i pdfen, kommer det tre newlines i txt-filen. Bytter ut disse med en enkelt newline slik at avsnitt får samme avstand som ellers i teksten.
Dersom det er følger liten bokstav etter et slik sideskift (leter etter to newlines etterfulgt av liten bokstav), fjernes newline slik at det ikke blir laget ett ekstra avsnitt som følge av parsingen av sideskift.


In [4]:
import re
from tika import parser

singlepdf_path = DATA_PATH + '/pdfs_v2/FLYKN-2023-00031.pdf'

raw = parser.from_file(singlepdf_path)
tika_text = raw['content']

def tika_cleanup(tika_text):
    # remove all text after "Nemndas representanter"
    nemndas_representanter_index = tika_text.find("Nemndas representanter")
    if nemndas_representanter_index != -1:
        tika_text = tika_text[:nemndas_representanter_index]

    # replace all occurrences of three or four consecutive newline characters with a single newline character
    tika_text = re.sub(r'\n\n\n', '\n', tika_text)

    # remove "\n"-characters if emmediately followed by a lowercase letter
    tika_text = re.sub(r'\n\n([a-zæøå])', r'\n\1', tika_text)

    tika_text = tika_text.strip()
    return tika_text

print(tika_cleanup(tika_text))

Vedtak i Transportklagenemnda - Fly

Sammendrag

Krav om erstatning for innkjøp som følge av forsinket bagasje.

Dato
12.09.2023

Saksnummer
2023-00031

Tjenesteytere
Air France

Klager har i det vesentlige anført

Klager reiste fra Oslo til Rio de Janeiro via Paris den 3. juli 2022 med Air France. Ved
ankomst Rio de Janeiro var ikke klagers koffert ankommet. 

Klager skulle delta i et bryllup, og i kofferten lå blant annet en dress. Klager ventet
helt til bryllupsdagen i påvente av bagasjen, før han måtte kjøpe seg en dress, pris
brasilianske reals (BRL) 4 743. Klager måtte også gjøre andre nødvendige innkjøp som
klær, toalettartikler og ny koffert, pris BRL 1 750. Da klager skulle på dagstur i regnskogen
måtte han også kjøpe idrettstøy som tålte en strabasiøs tur, pris BRL 1 269. Klager
måtte også kjøpe badetøy, badehåndkle og andre toalettartikler som klager ikke har
kvitteringer for.  

Bagasjen kom ikke frem til Brasil i de to ukene klager var der. Klager reiste hjem til Norge
den

Gjør om alle pdfer til txt-filer dersom de ikke allerede finnes. Om man vil lage txt-filene på ny kan man kommentere inn linjen som sletter txt-filene eller slette mappen manuelt.

In [5]:
PDFS_PATH = DATA_PATH + 'pdfs_v2/'
TXT_PATH = DATA_PATH + 'txts/'

In [6]:
from tika import parser
import shutil

pdfs_dir = PDFS_PATH
txts_dir = TXT_PATH

#shutil.rmtree(txts_dir, ignore_errors=True)

if not os.path.exists(txts_dir):
    os.mkdir(txts_dir)
    for f in os.listdir(pdfs_dir):
        raw = parser.from_file(os.path.join(pdfs_dir, f))
        tika_text = raw['content']
        tika_text = tika_cleanup(tika_text)
        with open(os.path.join(txts_dir, f.replace(".pdf", ".txt")), "w") as f:
            f.write(tika_text)

### Teller txt-filer

Viser antall txt-filer som er konvertert fra pdf og sorter dem etter kategori.

In [7]:
import pandas as pd



counts = {}

# loop through all the files in the directory
for file in os.listdir(TXT_PATH):
    name_code, year, *_ = file.split("-")
    counts[(name_code, year)] = counts.get((name_code, year), 0) + 1

df = pd.DataFrame.from_dict(counts, orient="index", columns=["Count"])

# create separate columns for year and name code
df[["Category", "Year"]] = pd.DataFrame(df.index.tolist(), index=df.index)

df['Category'] = df['Category'].replace({'FLYKN': 'Fly', 'KOLLKN': 'Kollektiv', 'PRKN': 'Pakkereise', 'SJTKN': 'Sjø'})

# pivot the table to have names as rows and years as columns
table = df.pivot(index="Year", columns="Category", values="Count")

table = table.fillna(0).astype("int")
table = table.sort_index(ascending=False)
table = table.reindex(columns=['Pakkereise', 'Fly', 'Sjø', 'Kollektiv'])

table['Total'] = table.sum(axis=1)
table.loc['All'] = table.sum()

# print the table
print(table)

Category  Pakkereise   Fly  Sjø  Kollektiv  Total
Year                                             
2023              81   106    7         64    258
2022              84   813   15        126   1038
2021              34   424   20        196    674
2020             176   727   17        209   1129
2019             141   863   23        239   1266
2018             243  1044    8        217   1512
2017             211   847   14        184   1256
2016             110   603    0         58    771
All             1080  5427  104       1293   7904


### Leser inn data

In [8]:
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader(TXT_PATH, glob="**/*.txt", loader_cls=TextLoader, use_multithreading=True)

documents = loader.load()

In [9]:
len(documents)

7904

In [10]:
documents[0]

Document(page_content='Vedtak i Transportklagenemnda - Fly\n\nSammendrag\n\nKrav om standarderstatning grunnet kansellering.\n\nDato\n14.02.2020\n\nSaksnummer\n2019-01569\n\nTjenesteytere\nWiderøe\n\nKlager har i det vesentlige anført\n\nKlager skulle reise fra Oslo kl. 12:10 til Florø med ankomst kl. 13:20 den 16. januar 2019\nmed WF135. Denne flyvningen ble først forsinket, og til slutt innstilt grunnet personell-\nmangel. Klager ble booket om til SK273 kl. 16:10 fra Oslo til Bergen og så videre med\nWF110 fra Bergen til Florø med ankomst Florø kl. 18:20\n\nKlager hadde en full flex billett og ønsket å benytte innlandsloungen på OSL, men fikk\nbeskjed om at siden han hadde en Widerøe-billett så var ikke det mulig. Klager reagerer\npå at det reklameres med at man har tilgang til loungen når det ikke er tilfellet. \n\nKlager krever standarderstatning på 250 euro i henhold til EU-forordningen 261/2004.\n\nTjenesteyterne har i det vesentlige anført\nI dette tilfellet skapte tett snøfall 

### Chunking

Bruker tiktoken tokenizer til å splitte tekstene i chunks, som er den samme tokenizeren openai bruker i modellene sine. Definerer hvor mange tokens det skal være i hver chunk ved å sette `chunk_size` variabelen. `chunk_overlap` definerer hvor mange tokens som skal overlappe mellom hver chunk. Dette gjør at modellen kan se sammenhengen mellom chunkene og vi passer på at vi ikke mister informajson der en chunk ville splittet f.eks. en setning i to.

Mindre chunks kan gi bedre søkeresultater på grunn av at hver chunk da vil inneholde mer spesifikk semantisk mening. Trade-offen er at det fort blir mange flere chunks å søke i. 500 tokens per chunk ser ut til å være en grei balanse etter å ha testet litt og fått gode og relevante dokumenter fra søket.

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=500, chunk_overlap=100)

texts = text_splitter.split_documents(documents)

In [12]:
len(texts)

37765

### Embedding

Lager embeddings på chunkene med OpenAI's ada-002. Dette gir oss en vektor for hver chunk som representerer meningen i chunken. Vi kan da sammenligne chunks med en vektor av spørringen for å finne de mest relevante chunkene.

Å sette `"device": "mps"` gjør at vi bruker GPUen til å hjelpe til med embedding på Mac.

Vi passer også på at vi ikke lager nye embeddings dersom vi allerede har en vectordatabase som er persistet. Da laster vi inn den istedenfor.

In [13]:
from langchain.embeddings import OpenAIEmbeddings


def getEmbeddings(texts):
    embedding = OpenAIEmbeddings(model_kwargs={"device": "mps"})
    persist_directory = "./vectordb"

    if persist_directory is not None and os.path.exists(persist_directory):
        vectordb = Chroma(embedding_function=embedding, persist_directory=persist_directory)
    else:
        vectordb = Chroma.from_documents(
            documents=texts, embedding=embedding, persist_directory=persist_directory
        )
        vectordb.persist()

    return vectordb

In [14]:
vectordb = getEmbeddings(documents)

Ser på antall chunks i vectordatabasen.

In [15]:
len(vectordb.get()["documents"])

37765

## Søk i embeddings

### Egendefinert VectorStoreRetriever

Retriever fra langchain vil returnere en liste med top *k* antall chunks som matcher søke-vektoren. For å kunne gi noe annet enn  selve chunken til modellen, har vi laget en egendefinert VectorStoreRetriever som kan returnere hele dokumenter i tillegg til andre deler av hele dokumenter. Ved å lage denne selv får vi ekstra fleksibilitet til å behandle kildene slk vi ønsker før de sendes til modellen.

Vi har lagt til funksjonalitet for å filtrere vekk dokumenter basert på hvilken nemnd som har behandlet saken, hvilket år saken er fra, og basert på enkelt saksnummer.
I tillegg kan vi legge til ekstra metadata som om hele saken som ble funnet i søket har en mindretallsuttalelse eller ikke, og om avgjørelsen blir fulgt opp av tjenesteyter eller ikke. Dette kan også filtreres på.

Ettersom vi filtrer bort mange søker, har vi lagt til muligheten til å definere både hvor mange top chunker man skal gjøre søk i (`k`), i tillegg til max dokumenter som skal returneres og brukes som kontekst til modellen (`max_elements`).

In [16]:
from langchain.vectorstores.base import VectorStoreRetriever
from langchain.vectorstores import VectorStore
from langchain.schema.retriever import Document
from typing import List
from langchain.pydantic_v1 import Field
from langchain.document_transformers import LongContextReorder
from langchain.document_loaders import TextLoader
import copy


class CustomRetriever(VectorStoreRetriever):
    """
    A custom retriever that extends the VectorStoreRetriever class.
    It retrieves relevant documents based on a query and filters them based on specified criteria.

    Args:

        vectorstore (VectorStore): The vectorstore to retrieve documents from.
        base_path (str): The base path of the documents.
        search_type (str): The type of search to perform. Can be "similarity" or "mmr".
        retrieve_type (str): The type of retrieval to perform. Can be "chunk", "parent" or "custom".
        max_elements (int): The maximum number of documents to retrieve.
        filter (dict): A dictionary specifying the filtering criteria with the following keys:\n
            "casetype": A list of case types to include in the filtered list. Eks.: ["FLYKN", "KOLLKN", "PRKN", "SJTKN"].\n
            "year": A list of years to include in the filtered list. Eks.: [2016, 2017, 2018, 2019, 2020, 2021, 2022].\n
            "exclude_case": A list of case numbers to exclude from the filtered list. Eks.: ["2022-00365", "2017-00001"]\n
            "har_mindretall": A boolean indicating whether to include only documents with dissenting opinions. Eks.: True\n
            "tjenesteyter_avviser": A boolean indicating whether to include only documents where the service provider does not comply with the decision. Eks.: True\n
        search_kwargs (dict): A dictionary specifying the search arguments. The dictionary can contain the following keys:
            "k": The number of documents to retrieve from the vectorstore.
    """

    vectorstore: VectorStore
    base_path: str = ""
    search_type: str = "similarity"
    retrieve_type: str = "chunk"
    max_elements: int = 4
    filter: dict = Field(default_factory=dict)
    search_kwargs: dict = Field(default_factory=dict)

    def get_relevant_documents(self, query: str) -> List[Document]:
        """
        Retrieves relevant documents based on a query and filters them based on specified criteria.

        Args:
            query (str): The query to retrieve relevant documents for.

        Returns:
            List[Document]: A list of `Document` objects that satisfy the filtering criteria.
        """
        chunks, scores = zip(
            *self.vectorstore.similarity_search_with_relevance_scores(
                query=query, **self.search_kwargs
            )
        )
        [
            chunk.metadata.update({"score": score})
            for chunk, score in zip(chunks, scores)
        ]

        result_docs = copy.deepcopy(chunks)
        parent_docs = self.get_parent_docs(chunks)

        if self.retrieve_type == "chunk":
            pass
        elif self.retrieve_type == "parent":
            result_docs = parent_docs
        elif self.retrieve_type == "custom":
            result_docs = self.custom_retrieve(parent_docs)

        result_docs = self.add_metadata(result_docs, parent_docs)
        result_docs = self.filter_docs(result_docs, self.filter)

        if self.retrieve_type == "chunk":
            result_docs = result_docs[: self.max_elements]
        else:
            # Retrieve new list of documents, where each source is only represented once.
            new_result_docs = []
            # Keep track of which sources have been added to the new list, and what position they have in the new_result_docs.
            added_sources = {}
            for i, doc in enumerate(result_docs):
                if doc.metadata["source"] not in added_sources.keys():
                    doc.metadata["chunk_count"] = 1
                    new_result_docs.append(doc)

                    added_sources[doc.metadata["source"]] = i
                    if len(new_result_docs) == self.max_elements:
                        break
                else:
                    new_result_docs[added_sources[doc.metadata["source"]]].metadata[
                        "chunk_count"
                    ] += 1

            result_docs = new_result_docs

        reordering = LongContextReorder()
        reordered_docs = reordering.transform_documents(result_docs)
        return reordered_docs

    def get_parent_docs(self, chunks: List[Document]):
        """
        Retrieves the parent documents for a list of `Document` objects.

        Args:
            docs (List[Document]): The list of `Document` objects to retrieve parent documents for.

        Returns:
            List[Document]: A list of `Document` objects that are the parent documents of the input `Document` objects.
        """
        parent_docs = []
        for chunk in chunks:
            text_loader = TextLoader(self.base_path + chunk.metadata["source"])

            parent_doc = text_loader.load()[0]
            parent_doc.metadata = copy.deepcopy(chunk.metadata)

            parent_docs.append(parent_doc)

        return parent_docs

    def add_metadata(slef, docs: List[Document], parent_docs: List[Document]):
        """
        Adds metadata to a list of `Document` objects based on their parent documents.

        Args:
            docs (List[Document]): The list of `Document` objects to add metadata to.
            parent_docs (List[Document]): The list of parent `Document` objects.

        Returns:
            List[Document]: A list of `Document` objects with added metadata.
        """
        for doc in docs:
            parent_doc = [
                parent_doc
                for parent_doc in parent_docs
                if parent_doc.metadata["source"] == doc.metadata["source"]
            ][0]
            doc.metadata["har_mindretall"] = (
                "mindretall" in parent_doc.page_content.lower()
            )
            doc.metadata["tjenesteyter_avviser"] = (
                "tjenesteyter følger ikke vedtaket i saken"
                in parent_doc.page_content.lower()
            )

        return docs

    def custom_retrieve(self, parent_docs: List[Document]):
        """
        Retrieves a custom set of documents based on their parent documents.

        Args:
            parent_docs (List[Document]): The list of parent `Document` objects to retrieve custom documents for.

        Returns:
            List[Document]: A list of `Document` objects that are a custom set of documents based on their parent documents.
        """
        results = copy.deepcopy(parent_docs)
        for doc in results:
            cutoff = doc.page_content.find("Nemnda bemerker")
            doc.page_content = doc.page_content[cutoff:]

        return results

    def filter_docs(self, docs: List[Document], filter: dict) -> List[Document]:
        """
        Filters a list of `Document` objects based on a set of criteria specified in a dictionary.

        Args:
            docs (List[Document]): The list of `Document` objects to filter.
            filter (dict): A dictionary specifying the filtering criteria. The dictionary can contain the following keys:
                - "casetype": A list of case types to include in the filtered list.
                - "year": A list of years to include in the filtered list.
                - "exclude_case": A list of case numbers to exclude from the filtered list.
                - "har_mindretall": A boolean indicating whether to include only documents with dissenting opinions.
                - "tjenesteyter_følger_ikke": A boolean indicating whether to include only documents where the service provider does not comply with the decision.

        Returns:
            List[Document]: A list of `Document` objects that satisfy the filtering criteria.
        """
        filtered_docs = []
        for doc in docs:
            doc.metadata["source"] = doc.metadata["source"].split("/")[-1]
            casetype, year, number = doc.metadata["source"].split(".")[0].split("-")

            if "casetype" in filter and casetype not in filter["casetype"]:
                continue

            if "year" in filter and int(year) not in filter["year"]:
                continue

            if (
                "exclude_case" in filter
                and f"{year}-{number}" in filter["exclude_case"]
            ):
                continue

            if (
                "har_mindretall" in filter
                and filter["har_mindretall"] != doc.metadata["har_mindretall"]
            ):
                continue

            if (
                "tjenesteyter_avviser" in filter
                and filter["tjenesteyter_avviser"]
                != doc.metadata["tjenesteyter_avviser"]
            ):
                continue

            filtered_docs.append(doc)

        return filtered_docs


Setter opp en retriever med de parametrene vi ønsker å bruke. `k=100` og `max_elements=10` gjør at vi søker opp topp 100 chunks, gjennomfører filtreringen og returnerer topp 10.

Setter `retrieve_type="chunk"` for å returnere chunkene som kontekst til modellen.
Kan også bruke `"parent"` for å returnere orignaldokumentene istedenfor bare chunkene eller `"custom"` for å returnere en annen del av dokumentet.

In [17]:
k = 100
max_elements = 10
test_custom_retriever = CustomRetriever(
    vectorstore=vectordb,
    retrieve_type="chunk",
    max_elements=max_elements,
    filter={},
    search_kwargs={
        "k": k,
    },
)

# FILTER_EXAMPLE = {
#       "casetype": ["FLYKN", "KOLLKN", "PRKN", "SJTKN"],
#       "year": [2016, 2017, 2018, 2019, 2020, 2021, 2022],
#       "exclude_case": ["2022-00365", "2017-00001"],
#       "har_mindretall": True,
#       "tjenesteyter_avviser": True
# }

### Eksempelsøk

Tester ut noe av funksjonaliteten til den egendefinerte CustomRetriever-klassen.

Sjekker først hva retrieveren returnerer.

In [18]:
query = "Får jeg dekket utgifter til taxi når bussen jeg skulle tatt ikke kunne komme på grunn av trafikkulykke?"

docs = test_custom_retriever.get_relevant_documents(query)
print(len(docs))
docs = sorted(docs, key=lambda x: x.metadata["score"], reverse=True)    # Sorterer dokumentene etter score
for doc in docs:
    print (doc.metadata)

10
{'source': 'KOLLKN-2023-00394.txt', 'score': 0.8408698530000864, 'har_mindretall': True, 'tjenesteyter_avviser': False}
{'source': 'KOLLKN-2023-00141.txt', 'score': 0.8403601395509077, 'har_mindretall': False, 'tjenesteyter_avviser': False}
{'source': 'KOLLKN-2018-00368.txt', 'score': 0.8384630361424475, 'har_mindretall': False, 'tjenesteyter_avviser': False}
{'source': 'KOLLKN-2022-00167.txt', 'score': 0.8364546334437821, 'har_mindretall': False, 'tjenesteyter_avviser': False}
{'source': 'KOLLKN-2017-02577.txt', 'score': 0.8345024019574696, 'har_mindretall': True, 'tjenesteyter_avviser': False}
{'source': 'KOLLKN-2023-00394.txt', 'score': 0.8342651573472019, 'har_mindretall': True, 'tjenesteyter_avviser': False}
{'source': 'KOLLKN-2023-01036.txt', 'score': 0.8336283711497628, 'har_mindretall': False, 'tjenesteyter_avviser': False}
{'source': 'KOLLKN-2023-01200.txt', 'score': 0.8313629675055902, 'har_mindretall': False, 'tjenesteyter_avviser': False}
{'source': 'KOLLKN-2023-01559.tx

#### `retrieve_type="parent"`

Tester ut `get_parent_docs()` fra customRetrieveren.

Denne Henter ut hele dokumentet der chunken kommer fra. Dersom søket returnerer flere chunks fra samme originaldokument, vil hele dokumentet bli returnert flere ganger. Disse vil bli filtrert bort i `get_relevant_documents()` og legges til i en teller `chunk_count` dersom `retrieve_type` blir satt til `"parent"` eller `"custom"`.

In [19]:
parent_docs = []
for doc in docs:
    text_loader = TextLoader(TXT_PATH + doc.metadata["source"])

    parent_doc = text_loader.load()[0]
    parent_doc.metadata = copy.deepcopy(doc.metadata)

    parent_docs.append(parent_doc)

parent_docs

[Document(page_content='Vedtak i Transportklagenemnda\n- Kollektivreiser\n\nSammendrag\n\nKrav om at drosjeutgifter dekkes som følge av at bussen\nikke kom grunnet snøvær. Anført misvisende informasjon.\n\nDato\n30.05.2023\n\nSaksnummer\n2023-00394\n\nTjenesteytere\nRuter\n\nKlager har i det vesentlige anført\n\nKlager skulle ta buss 79 fra Ljabru stasjon om ettermiddagen den 17. desember 2022.\nKlager anfører at det på lystavlen sto at bussen var på vei. Klager anfører at det på\nlystavlen sto at bussen skulle komme om 14 minutter, men etter 14 minutter forsvant\ninformasjonen om at bussen skulle komme nå, og bussen dukket ikke opp. Klager\nanfører at det deretter sto at bussen skulle komme om seks minutter, men denne\ninformasjonen forsvant igjen seks minutter etter. Klager anfører at det deretter kom opp\ninformasjon om at bussen skulle komme om 13 minutter, men heller ikke denne gangen\ndukket det opp noen buss. Klager anfører at alle tidene deretter forsvant fra tavlen, og\netter 

Tester funksjonen  legger til en verdi i metadatafeltet som sier noe om hvor mange chunks som ga treff på hvert dokument som ble returnert til modellen.

In [20]:
result_docs = []
# Keep track of which sources have been added to the new list, and what position they have in the new_result_docs.
added_sources = {}
for i, doc in enumerate(parent_docs):
    if doc.metadata["source"] not in added_sources.keys():
        doc.metadata["chunk_count"] = 1
        result_docs.append(doc)

        added_sources[doc.metadata["source"]] = i
        if len(result_docs) == max_elements:
            break
    else:
        result_docs[added_sources[doc.metadata["source"]]].metadata[
            "chunk_count"
        ] += 1

for doc in result_docs:
    print (doc.metadata)

{'source': 'KOLLKN-2023-00394.txt', 'score': 0.8408698530000864, 'har_mindretall': True, 'tjenesteyter_avviser': False, 'chunk_count': 2}
{'source': 'KOLLKN-2023-00141.txt', 'score': 0.8403601395509077, 'har_mindretall': False, 'tjenesteyter_avviser': False, 'chunk_count': 1}
{'source': 'KOLLKN-2018-00368.txt', 'score': 0.8384630361424475, 'har_mindretall': False, 'tjenesteyter_avviser': False, 'chunk_count': 1}
{'source': 'KOLLKN-2022-00167.txt', 'score': 0.8364546334437821, 'har_mindretall': False, 'tjenesteyter_avviser': False, 'chunk_count': 1}
{'source': 'KOLLKN-2017-02577.txt', 'score': 0.8345024019574696, 'har_mindretall': True, 'tjenesteyter_avviser': False, 'chunk_count': 1}
{'source': 'KOLLKN-2023-01036.txt', 'score': 0.8336283711497628, 'har_mindretall': False, 'tjenesteyter_avviser': False, 'chunk_count': 1}
{'source': 'KOLLKN-2023-01200.txt', 'score': 0.8313629675055902, 'har_mindretall': False, 'tjenesteyter_avviser': False, 'chunk_count': 1}
{'source': 'KOLLKN-2023-01559

#### `retrieve_type="custom"`

Tester muligheten for å bare hente ut deler av dokumentet. Det er ikke sikkert at man trenger å gi fullstendige saker til modellen som kontekst. Dette kan potensielt gi veldig mange tokens som modellen må håndtere.
Ser om vi får et godt resultat av å søke på overskriften "Nemnda bemerker" i ett saksdukument og returnere alt som kommer etter dette. Siden dete er siste overskrift, burde det holde å sortere vekk all tekst som kommer før og beholde alt etter.

In [21]:
result = copy.deepcopy(result_docs)
for i, doc in enumerate(result_docs):
    cutoff = doc.page_content.find("Nemnda bemerker")
    result[i].page_content = doc.page_content[cutoff:]

result

[Document(page_content='Nemnda bemerker\n\nKlager har fremsatt krav om at utgifter til drosje dekkes som følge av at bussen hun\nskulle ta fra Ljabru stasjon i 17-tiden den 17. desember 2022 ikke dukket opp.\n\nDet fremgår av reisegarantien til Ruter at dersom passasjeren blir mer enn 20 minutter\nforsinket som følge av at Ruter ikke er i rute, dekker Ruter dokumenterte utlegg på inntil\n750 kroner dersom vilkårene for dette er oppfylt. Garantien gjelder imidlertid ikke hvis\nårsaken til forsinkelsen ikke skyldes Ruter, som ved kraftig snøfall som her.\n\nI dette tilfellet har Ruter forklart at trafikale problemer skapt av snøvær var omtalt i flere\naviser i forkant. Nemndas flertall, nemndselder og bransjerepresentantene, finner ikke\ngrunn til å tvile på at snøværet var årsak til at bussen ikke kom. Det vises blant annet\ntil meldingen fra operatør som tjenesteyter har vist til i sitt tilsvar, om at det ikke var\ntrygt å kjøre i området klager befant seg. Flertallet mener dermed at r

## Sette opp QA chain

Laster inn openAI sin gpt-modell og setter opp en pipeline for å kjøre spørringer mot modellen. Modellnavnet kan endres til f.eks.: "gpt-4" for å bruke denne i steden.

Setter temperaturen til 0.0. Enkelt forklart sier denne noe om hvor kreativ modellen skal være. 0.0 betyr at den skal være helt deterministisk og gi samme svar hver gang. Dette er viktig for å kunne sammenligne resultater fra forskjellige søk.

In [114]:
llm = ChatOpenAI(model="gpt-4", temperature=0.0)

Definerer en prompt som forteller hvordan llm-modellen skal oppføre seg og svare på spørsmålene. Her kan det eksperimenteres med ulike oppsett for å se hva som gir best resultat.

Prompten som ligger inne her er en veldig enkel variant. 

In [129]:
from langchain.prompts import PromptTemplate

template = """Always answer as helpfully as possible using the context cases provided. \
        If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\
        If you don't know the answer to a question, please don't share false information.
        Always answer in the same language as the question. (Default language is Norwegian)

        Use these instruction steps when answering questions:

        **Steps:**
                1. Retrieve the jurisdictional arguments from the context cases.
                2. pick out the arguments that are most relevant to the question.
                3. Use these to build up your answer.
                4. If there is no clear answer based on previous cases. Use examples instead and explain why the answer is not clear.

        Historical cases: {context}

        Question: {question}
        
        Answer:"""

QA_CHAIN_PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])

In [130]:
chain_type_kwargs = {"prompt": QA_CHAIN_PROMPT}

Dersom man bruker `"parent"` som retrieve type, kan det hende at antall tokens blir for mye for modellen. Det kan derfor være lurt å sette `max_elements` til 5 eller mindre.

In [131]:
k = 100
max_elements = 4
custom_retriever = CustomRetriever(
    vectorstore=vectordb,
    retrieve_type="parent",
    max_elements=max_elements,
    filter={},
    search_kwargs={
        "k": k,
    },
)

In [132]:
# create the chain to answer questions 
qa_chain = RetrievalQA.from_chain_type(llm=llm, 
                                  chain_type="stuff", 
                                  retriever=custom_retriever, 
                                  chain_type_kwargs=chain_type_kwargs, 
                                  return_source_documents=True)

### Formater resultat

In [133]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    llm_response["source_documents"] = sorted(llm_response["source_documents"], key=lambda x: x.metadata["score"], reverse=True)
    for i, source in enumerate(llm_response["source_documents"]):
        output = f"{i+1}:\t{source.metadata['source']} | score: {str(format(source.metadata['score'], '.4f'))} | har_mindretall: {str(source.metadata['har_mindretall'])}"
        if 'chunk_count' in source.metadata.keys():
            output += f" | chunk_count: {str(source.metadata['chunk_count'])}"
        print(output)

### Kjører modellen

In [134]:
question = "Får jeg dekket utgifter til taxi når bussen jeg skulle tatt ikke kunne komme på grunn av trafikkulykke?"

In [135]:
llm_response = qa_chain(question)

In [136]:
process_llm_response(llm_response)

Basert på tidligere saker fra Transportklagenemnda, vil det avhenge av flere faktorer. Hvis bussen er mer enn
20 minutter forsinket og dette skyldes forhold som ligger innenfor transportørens kontroll, kan du ha rett til
å få dekket dokumenterte utgifter til alternativ transport, som for eksempel taxi, opp til et visst beløp.

I en sak fra 2023 (saksnummer 2023-00141) ble det for eksempel ikke gitt dekning for drosjeutgifter da bussen
ikke kom på grunn av snøvær, ettersom dette ble ansett som et forhold utenfor transportørens kontroll.

I en annen sak fra 2018 (saksnummer 2018-00368) ble det heller ikke gitt dekning for drosjeutgifter da bussen
var forsinket på grunn av trafikkforhold, ettersom dette også ble ansett som et forhold utenfor transportørens
kontroll.

Hvis en trafikkulykke er årsaken til at bussen ikke kommer, vil det trolig bli ansett som et forhold utenfor
transportørens kontroll, og du vil derfor ikke ha rett til å få dekket utgifter til taxi. Men dette vil
avhenge av d

### Viser kilder

In [128]:
from IPython.display import Markdown, display

result_md = f"## Sources: \n"
for doc in llm_response["source_documents"]:
    result_md += "**Source: " + doc.metadata['source'] + "** \n\n" + doc.page_content + "\n\n --- \n"

md_doc = Markdown(result_md)
display(md_doc)

## Sources: 
**Source: KOLLKN-2023-00394.txt** 

Vedtak i Transportklagenemnda
- Kollektivreiser

Sammendrag

Krav om at drosjeutgifter dekkes som følge av at bussen
ikke kom grunnet snøvær. Anført misvisende informasjon.

Dato
30.05.2023

Saksnummer
2023-00394

Tjenesteytere
Ruter

Klager har i det vesentlige anført

Klager skulle ta buss 79 fra Ljabru stasjon om ettermiddagen den 17. desember 2022.
Klager anfører at det på lystavlen sto at bussen var på vei. Klager anfører at det på
lystavlen sto at bussen skulle komme om 14 minutter, men etter 14 minutter forsvant
informasjonen om at bussen skulle komme nå, og bussen dukket ikke opp. Klager
anfører at det deretter sto at bussen skulle komme om seks minutter, men denne
informasjonen forsvant igjen seks minutter etter. Klager anfører at det deretter kom opp
informasjon om at bussen skulle komme om 13 minutter, men heller ikke denne gangen
dukket det opp noen buss. Klager anfører at alle tidene deretter forsvant fra tavlen, og
etter mer enn én times venting valgte klager å ta drosje.

Klager anfører at hun ikke visste eller hadde mulighet til å vite om bussen hadde
problemer på veien, fordi Ruters egen lystavle opplyste om at bussen var på vei til Ljabru
holdeplass. Klager anfører at lystavlen ga henne all grunn til å vente på bussen. Klager
anfører at hun ble stående og vente, fordi Ruter gjentatte ganger opplyste om at bussen
kom. Klager anfører at Ruter ga henne feil informasjon og falske løfter om bussankomst.
Klager anfører at hun ikke hadde mulighet til å vite at hun ble villedet av Ruter og at de
ga henne feil informasjon som medførte at hun sto og ventet og frøs i kulden i nærmere
én time.

Klager anfører at Ruter lover å dekke drosje i et slikt tilfelle i henhold til sin reisegaranti.

Til Ruters tilsvar anfører klager blant annet at nedtellingen på lystavlen viste at
bussen var på vei, men trolig forsinket, og at forsinkelse som det ikke opplyses om
dekkes av Ruters reisegaranti. Klager anfører at det ikke var meldt til dem som sto på
bussholdeplassen kl. 17 den dagen at det ikke var forsvarlig å kjøre. Klager anfører at det
var kaldt og litt glatt, men det snødde ikke, og det var ikke «ekstreme værforhold». Klager
anfører at en buss som gikk motsatt vei, gikk som normalt, og sjåføren oppga at bussen
var forsikret. Klager anfører at hvis alle Ruters kunder skulle la være å ta trikk og buss
hver gang det snør, ville det oppstått kaos, og det er verken mulig eller ønskelig.

Klager krever at Ruter dekker drosjeutgiftene på 600 kroner.

Tjenesteyterne har i det vesentlige anført
Den 17. desember 2022 var en av store «snødagene» som var godt dekket i alle landets
aviser i forkant. Det ble frarådet å reise om man ikke måtte, og det ble også anbefalt å ha
hjemmekontor for de som hadde mulighet til det. Dette påvirket tilbudet til Ruter i form
av forsinkelser og innstilte avganger for buss, trikk og T-bane. Noen steder var det ikke
mulig å kjøre, andre steder var det ikke forsvarlig.

I Ruters reisegaranti står det som følger: «Du får ikke refusjon hvis du visste om eller
burde ha visst om problemet på forhånd, eller hvis årsaken ikke ligger hos oss. Dette kan
blant annet omfatte følgende forhold: Ekstreme værforhold som kraftig vind, kraftig regn
og kraftige snøfall.»

Om sanntidstavlene på holdeplassen teller ned og bussen ikke kommer, kan det tyde på
at bussen var forsinket eller innstilt.

Dette ble meldt til Ruter denne dagen. I tillegg fikk Ruter denne meldingen fra operatør
som drifter linje 79 kl. 15.30 den 17.12: «Det er ikke forsvarlig å kjøre nå på grunn av
mengde av snøfall. På grunn av sikkerhet ønsker jeg å stoppe alle trafikk i følgende
områder fram til en vurdering at det er trygt å kjøre igjen: Bjørndal, Holmlia, Tårnåsen og
Grorudveien».

På dager som den 17. desember 2022 var høyeste prioritet å ha kontroll på trafikken. Da
kan det ta noe tid å få lagt inn ny informasjon eller fjerne en avgang i sanntid.

På grunnlag av dette avviser Ruter kravet om at drosjeutgiftene dekkes.

Nemnda bemerker

Klager har fremsatt krav om at utgifter til drosje dekkes som følge av at bussen hun
skulle ta fra Ljabru stasjon i 17-tiden den 17. desember 2022 ikke dukket opp.

Det fremgår av reisegarantien til Ruter at dersom passasjeren blir mer enn 20 minutter
forsinket som følge av at Ruter ikke er i rute, dekker Ruter dokumenterte utlegg på inntil
750 kroner dersom vilkårene for dette er oppfylt. Garantien gjelder imidlertid ikke hvis
årsaken til forsinkelsen ikke skyldes Ruter, som ved kraftig snøfall som her.

I dette tilfellet har Ruter forklart at trafikale problemer skapt av snøvær var omtalt i flere
aviser i forkant. Nemndas flertall, nemndselder og bransjerepresentantene, finner ikke
grunn til å tvile på at snøværet var årsak til at bussen ikke kom. Det vises blant annet
til meldingen fra operatør som tjenesteyter har vist til i sitt tilsvar, om at det ikke var
trygt å kjøre i området klager befant seg. Flertallet mener dermed at reisegarantien
ikke gjelder i dette tilfellet, og at klager ikke har rett på å få dekket utgiftene til drosje.
Flertallet vil imidlertid bemerke at det på lystavlen med avgangene burde vært opplyst
om forsinkelser og innstillinger.

Mindretallet, forbrukerrepresentantene, bemerker at 20 minutters forsinkelse som
hovedregel gir rett til erstatning for alternativ transport i henhold til Ruters reisegaranti.
Under garantien er det tilknyttet enkelte unntak, og tjenesteyter har bevisbyrden for at
disse er oppfylt. I foreliggende sak er klagers krav avslått med begrunnelse i værforhold,
og at passasjeren «visste om eller burde ha visst problemet på forhånd».

Når det gjelder unntaket «burde visst», er det etter mindretallets syn ikke tilstrekkelig at
snøværet var omtalt i aviser. Omtalen måtte eventuelt danne tilstrekkelig grunnlag for
at den konkrete reisen ville rammes. Ruter har ikke fremlagt dokumentasjon for dette.
Med kunnskap om at værforholdene berører konkrete ruter til spesielle tidspunkter vil
dette utvide tjenesteyters opplysningsplikt. En profesjonell tjenesteyter har den tekniske
kompetansen til å kunne oppdatere konsekvenser av ekstreme værforhold i sanntid.
Ved vurderingen av om den reisende «burde visst» om forsinkelsen må det ses hen til
både hvilken informasjon selskapet gir sine reisende og den reisendes egenrisiko for å
tilegne seg informasjon. Tjenesteyters opplysningsplikt vil normalt gå foran forbrukeres
undersøkelsesplikt. En reisende må kunne legge til grunn oppdaterte rutetider som gis
på sanntidstavler. Ytterlige forsinkelser bør den profesjonelle part bære risikoen for.

Den andre aktuelle unntaksregelen fordrer årsaker utenfor tjenesteyters kontroll, som
«ekstreme værforhold». Det forhold at det til tider kan komme mye regn eller snø på
en dag er ikke i seg selv ekstremt. I saken er det ikke fremlagt dokumentasjon på hvor
mye snøfall det var i det aktuelle området denne dagen. Tjenesteyter kan etter dette ikke
anses å ha oppfylt sin bevisbyrde.

Etter dette mener mindretallet at klager bør gis medhold i dekning av taxikostnadene på
600 kroner.

Vedtak

Transportklagenemnda kan ikke anbefale at klager gis medhold. Dissens.

 --- 
**Source: KOLLKN-2023-00141.txt** 

Vedtak i Transportklagenemnda
- Kollektivreiser

Sammendrag

Krav om at drosjeutgifter dekkes som følge av at bussen ikke kom grunnet snøvær.

Dato
30.05.2023

Saksnummer
2023-00141

Tjenesteytere
Ruter

Klager har i det vesentlige anført

Klager skulle ta bussen fra The Well til Oslo om kvelden den 26. desember 2022.

Klager anfører at bussen ikke dukket opp, og at han ventet mellom 40 og 60 minutter
på bussholdeplassen før han tok drosje. Klager anfører at det ikke var informert om
kanselleringen, og at det i Google Maps-appen sto at bussen var i rute.

Klager anfører at han ikke hadde kjøpt bussbillett på forhånd. Klager anfører at Ruter
ikke informerte ham på noen måte om uregelmessigheter, og at selskapet må finne en
måte å informere om slike ting via skjermer med ankomsttid slik de har andre steder.

Klager anfører at han ikke hadde noen annen måte å komme seg tilbake til byen på, slik
at eneste alternativ var å ta drosje.

Klager krever at Ruter dekker drosjeutgiftene på 898 kroner.

Tjenesteyterne har i det vesentlige anført
Den 26. desember 2022 var en av store «snødagene». Det ble skrevet om utfordringene
med snøværet i flere av landets aviser. Dette påvirket Ruters tilbud i form av forsinkelser
og innstilte avganger for buss, trikk og T-bane. Noen steder var det ikke mulig å kjøre, og
andre steder var det ikke forsvarlig.

I Ruters reisegaranti står det som følger: «Du får ikke refusjon hvis du visste om eller
burde ha visst om problemet på forhånd, eller hvis årsaken ikke ligger hos oss. Dette kan
blant annet omfatte følgende forhold: Ekstreme værforhold som kraftig vind, kraftig regn
og kraftige snøfall.»

På grunnlag av dette avslår Ruter kravet om dekning av drosjeutgiftene.

Nemnda bemerker

Klager har fremsatt krav om dekning av drosjeutgifter som følge av at bussen fra The
Well til Oslo om kvelden den 26. desember 2022 ikke dukket opp.

Det fremgår av reisegarantien til Ruter at dersom passasjeren blir mer enn 20 minutter
forsinket som følge av at Ruter ikke er i rute, dekker Ruter dokumenterte utlegg på inntil
750 kroner dersom vilkårene for dette er oppfylt. Garantien gjelder imidlertid ikke hvis
årsaken til forsinkelsen ikke skyldes Ruter, som ved kraftig snøfall som her.

I dette tilfellet har Ruter vist til at det var store trafikale problemer i Oslo den aktuelle
dagen, og har dokumentert at dette var omtalt i landets aviser. Nemnda mener at Ruter
ikke kan lastes for innstillingen på grunn av snøværet. Nemnda har ikke grunn til å tvile
på at været var grunnen til forsinkelsene og innstillingene denne dagen, og at bussen
ville ha kjørt dersom dette hadde vært forsvarlig.

Nemnda kan etter dette ikke se at klager har rett på å få dekket utgiftene til drosje.

Vedtak

Transportklagenemnda kan ikke anbefale at klager gis medhold.

 --- 
**Source: KOLLKN-2018-00368.txt** 

Vedtak i Transportklagenemnda
- Kollektivreiser

Sammendrag

Krav om refusjon av utgift til taxi som følge av forsinket buss.

Dato
18.04.2018

Saksnummer
2018-00368

Tjenesteytere
Nettbuss Travel AS

Klager har i det vesentlige anført

Klager tok bussen fra Sjølyst  til Fokserød 28. januar 2018 med avreise kl.15.20 og
planlagt ankomst på Fokserød kl.16.40. 

Klager skulle hjem til en familiemiddag kl.17.00 og hadde avtalt med sin søster at hun
skulle plukke opp klager ved Fokserød kl.16.40. 

På tidspunktet for opprinnelig ankomst på Fokserød befant bussen seg i Drammen.
Bussjåføren ga da beskjed om at det var forventet en forsinkelse på 45 minutter pga. tett
trafikk. Siden det ikke var mulig for klager å bli hentet etter kl.17.00 bestilte han en taxi
da bussen var ved Tønsberg. 

Etter klagers erfaring  var trafikken ikke noe utover det vanlige. Klager viser videre til
vedlagt oversikt fra Vegvesenet som viser normal trafikkavvikling den dagen. Det var
også relativt gode vei og føreforhold for sesongen med tørr veibane, ingen nedbør og
delvis overskyet vær. Klager viser her til vedlagt væroversikt fra YR.no.

Klagers oppfatning er at Nettbuss sine rutetider er misvisende siden selskapet ikke tar
hensyn til rushtrafikk mellom Oslo og Asker, samt ved Drammensbroen. Særlig siden
rushtrafikk er permanent fenomen mellom denne strekningen i dette tidsvinduet.
Hadde klager på forhånd hatt kjennskap til at en forsinkelse måtte påregnes, hadde han
benyttet seg av toget siden dette transportmiddelet ikke er befattet med akkurat denne
utfordringen. 

Klager krever derfor at Nettbuss refunderer utgiften til taxi på kr 206.

Tjenesteyterne har i det vesentlige anført

I denne saken viser Nettbuss til informasjonen som fremgår i Reisegarantien på
selskapets nettside. 

Reisegarantien gjelder ikke dersom forsinkelsen eller innstillingen skyldes forhold
utenfor transportørens kontroll, slik som offentlige påbud og forbud, streik og lignende,
naturkatastrofer, ekstraordinære værforhold (f. eks. kraftig snøfall, ras, flom eller
eksepsjonelt glatt veibane), veiarbeid eller uforutsette problemer langs veien, og
store arrangementer eller andre trafikale forhold som berører kollektivtrafikken i stor
grad. Den omfatter heller ikke følgeskader av forsinkelsen f. eks hvis du ikke rekker
tannlegetime, forretningsavtale eller en flyavgang.

Bussen ble forsinket underveis som følge av trafikale forhold på den aktuelle avgangen.
Reisende har imidlertid reist med bussen han hadde billett for og har dermed heller ikke
hatt utlegg for alternativ transport til transporten han hadde forhåndskjøpt.

Reisende sitt krav om dekt kostnad til taxi ved avstigning på Fokserød, fordi hans
avtale om videre privat transport da var falt bort som følge av at bussen ankom senere
enn forventet, er avslått da Reisegarantien uansett ikke dekker denne typen utlegg
(=følgeskader). Reisende er informert om sine rettigheter via Reisegarantien som det
linkes til i vilkårene han godkjenner ved kjøp av nettbillett.

På grunnlag av overnevnte kan ikke Nettbuss imøtekomme klagers krav om refusjon av
utgift til drosje.

Nemnda bemerker

Klager tok bussen fra Sjølyst  til Fokserød 28. januar 2018 med avreise kl.15.20 og
planlagt ankomst på Fokserød kl.16.40. Bussen ble ca 45 minutter forsinket og klager
krevet dekket utgifter til taxi med kr. 206.

Nettbuss har en reisegaranti som på visse vilkår dekker utgifter til alternativ transport
ved forsinkelser. 

Nemnda antar at reisegarantien i prinsippet omfatter en forsinkelse av den lengde det
her er tale om. 

Nettbuss hevder at reisegarantien ikke kommer til anvendelse og viser til følgende
bestemmelse i garantien: 

"Reisegarantien gjelder ikke dersom forsinkelsen eller innstillingen skyldes forhold
utenfor transportørens kontroll, slik som offentlige påbud og forbud, streik og lignende,
naturkatastrofer, ekstraordinære værforhold (f. eks. kraftig snøfall, ras, flom eller
eksepsjonelt glatt veibane), veiarbeid eller uforutsette problemer langs veien, og
store arrangementer eller andre trafikale forhold som berører kollektivtrafikken i stor
grad. Den omfatter heller ikke følgeskader av forsinkelsen f. eks hvis du ikke rekker
tannlegetime, forretningsavtale eller en flyavgang."

Nemnda viser til at følgeskader av forsinkelsen ikke omfattes av reisegarantien og kan
ikke se at klager har krav på å få erstattet utgiftene til taxi. 

Nemnda finner imidlertid grunn til å føye til at de trafikale forhold neppe var av en slik
karakter at de kan likestilles med force majure lignende forhold som nevnt ovenfor.
Det er ikke er gitt opplysninger om spesielle større hendelser i trafikken som årsak til
forsinkelsen, og går ut fra at det dreide seg om en forsinkelse som ofte oppstår på E-18.

Vedtak

Transportklagenemnda Kollektivtransport finner ikke å kunne anbefale at klager gis
medhold.

Vedtaket er enstemmig.

 --- 
**Source: KOLLKN-2022-00167.txt** 

Vedtak i Transportklagenemnda
- Kollektivreiser

Sammendrag

Krav om refusjon av avbestillingsgebyr hos frisør som følge av forsinkelse.

Dato
19.05.2022

Saksnummer
2022-00167

Tjenesteytere
Vy

Klager har i det vesentlige anført

Klager skulle ta toget inn til Oslo for å nå en frisørtime kl. 10.00 den 20. januar 2022.
Klager har anført at toget hennes skulle gå kl. 09.06 og være på Nationaltheatret kl.
09.40. Klager anfører at toget først var forsinket før det ankom Gardermoen, og da det
først kom, sto toget stille en god stund før det ble innstilt. Klager anfører at da toget ble
innstilt, var det for sent for henne å rekke frisørtimen med noe annet transportmiddel fra
Gardermoen. Klager anfører at hun i utgangspunktet hadde beregnet god tid til timen.
Klager anfører at hun måtte betale et gebyr for manglende oppmøte hos frisøren.

Klager anfører at hun har månedsbillett som hun bruker til og fra jobb hele året. Klager
oppgir at hun har mottatt 34 kroner i refusjon av månedsbilletten for reisen.

Klager påpeker at hun ikke valgte å avbestille frisørtimen selv, og at det var innstillingen
av toget som førte til at hun måtte betale 70 % av hva frisørtimen kostet. Klager anfører
at hun har måttet betale for en vare hun ikke har fått, i tillegg til at hun må betale for en
ny time. Klager viser til informasjon på Forbrukerrådets sider om rett til å få refusjon av
ekstra utgifter ved forsinkelser.

Klager anfører at det ikke fantes alternativ transport som gjorde at hun ville rekke avtalen
i dette tilfellet. Klager anfører videre at grunnen til at hun tar tog på denne strekningen,
er at det er det raskeste alternativet.

Det vises til klagers øvrige anførsler i fremlagt dokumentasjon.

Klager krever at frisørgebyret på 868 kroner dekkes som følge av innstillingen.

Tjenesteyterne har i det vesentlige anført

Klager reiste med tog 811 den 20. januar som hadde annonsert avgangstid fra Eidsvoll
Verk kl. 09.06 og annonsert ankomst på Nationaltheatret kl. 09.40. Toget avgikk noe
forsinket, kl. 09.10. Underveis, ved Oslo lufthavn, ble det imidlertid stopp når toget
beveget seg, da det oppstod feil på linjen, en sporveksel. Toget ble stående og ankom
Nationaltheatret kl. 10.57 – 77 minutter etter rutetid. Det vises til fremlagt oversikt over
tider for toget.

Bane Nor opplyser: «20. januar klokken 09:17 fikk vi feil på en veksel på Gardermoen,
som medførte store driftsforstyrrelser for en rekke tog. Feilen ble utbedret kl. 10:03, før
den kom tilbake igjen kort tid etter. Klokken 10:53 ble feilen reparert på nytt, og trafikken
gravis normalisert etter dette.»

Når det oppstår plutselige avvik som her, vil det være vanskelig å estimere hvor lenge
dette vil vare. Det kan ta alt fra minutter til dager å utføre feilsøking og retting. Avviket
må ha vedvart i noe tid før Vy arrangerer og annonserer alternativ transport til alle
berørte stasjoner, som vil ta timer å få på plass og er prisgitt at tilbydere kan stille med
ledig kapasitet. Den enkelte passasjer kan imidlertid selv benytte alternativ transport
som for eksempel buss og drosje i henhold til Vys reisevilkår punkt 6 hvor det er angitt
som følger:

«Dine rettigheter hvis toget blir forsinket eller innstilt
A. Vys reisegaranti gir deg rettigheter hvis ditt Vy tog er forsinket eller innstilt. Hvis
toget er forsinket eller innstilt, tilbyr Vy et alternativ. Det kan være neste tog eller annet
transportmiddel.
Dersom vi ikke har gitt deg informasjon om hvordan du skal komme deg videre innen
rimelig tid, kan du få dekket dokumenterte, påregnelige, direkte utgifter til alternativ
transport avhengig av reiselengde og forsinkelsens varighet:
• For forsinkelse på 20 minutter eller mer på reiser under 1 time kan du få dekket utgifter
opp til 550 kr.
• For forsinkelse på 40 minutter eller mer på reiser mellom 1 og 3 timer kan du få dekket
utgifter opp til 825 kr.
• For forsinkelse på 60 minutter eller mer på alle reiser inntil en grense på 30 prosent av
grunnbeløpet i folketrygden (G).»

Ved forsinkelse kan man altså få dekket utlegg til egen arrangert alternativ transport.
Hvorvidt klager hadde kommet frem i tide om hun hadde benyttet alternativ transport,
er uklart.

Dessverre ble tusenvis av reisende berørt av dette avviket som oppstod plutselig. Det
at klager pådro seg utlegg som følge av at hun ikke fikk benyttet seg av frisørtime, er
beklagelig, men dette er ikke en direkte påregnelig utgift jf. vilkårene nevnt ovenfor, men
en upåregnelig og avledet følgeskade, og det foreligger ikke grunnlag for dekning av et
slikt utlegg i henhold til vilkårene.

Vys reisevilkår omfatter ikke dekning av følgeskader med unntak av togreise til flyplass
der Vy konkret markedsfører dette. Dette er for øvrig i samsvar med bransjestandard for
kollektivtrafikk i Norge hvor det fremgår følgende: «Reisegarantien omfatter heller ikke
følgeskader av forsinkelsen f.eks. ved at du ikke rekker tannlegetime, forretningsavtale
eller en flyavgang.»

Vy anser klagers utlegg som et avledet tap og følgeskade som ikke er omfattet av Vys
vilkår, og Vy opprettholder avslaget om dekning. Klager har imidlertid fått refundert 50 %
av dagsverdien på sin periodebillett som en følge av forsinkelsen jf. reisevilkårene punkt
6 bokstav D.

Nemnda bemerker

På bakgrunn av klagers og tjenesteyters opplysninger legger nemnda til grunn at klager
har fått 50 % refusjon av dagsverdien av periodebilletten, i tråd med Vys reisevilkår punkt
6 D. Klager krever i tillegg refusjon av betaling for en frisørtime hun ikke rakk på grunn av
at toget ble forsinket.

Det fremgår av Vys reisegaranti at det ved forsinkelser og innstillinger kan gis refusjon
av dokumenterte, påregnelige, direkte utgifter til alternativ transport, avhengig av
reiselengde og forsinkelsens varighet. Garantien omfatter ikke følgeskader. Nemnda kan
derfor ikke se at klager har rett på refusjon av utgiften til ubenyttet frisørtime.

Vedtak

Transportklagenemnda kan ikke anbefale at klager gis medhold.

 --- 
